# Bistable ODE Simulation
This notebook simulates the bistable ODE:

$$
\frac{dy}{dt} = -r(y - 1) (2 - y) (y - 3) \\
y(0) = y_0
$$
where $r\sim \mathcal{U}(0.8, 1.2)$ and $y_0 \sim \mathcal{U}(0,4)$.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from IPython.display import HTML

# Define the bistable ODE
def bistable_ode(t, y, r):
    return - r * (1 - y) * (3 - y) * (y - 2)

Ns = 1000
r = np.random.uniform(0.8, 1.2, (Ns, 1))
y0 = np.random.uniform(0, 4, (Ns, 1))

parameters = np.hstack((y0, r))

t_span = (0, 8)
t_eval = np.linspace(t_span[0], t_span[1], 100)

results = []
for p in parameters:
    sol = solve_ivp(
        bistable_ode,
        t_span, [p[0]],
        t_eval=t_eval,
        args=(p[1],),
        method="Radau"
    )
    results.append(sol.y[0])
    
    if not sol.success:
        print(f"Integration failed for parameters: {p}, message: {sol.message}")
        results.append(np.full_like(t_eval, np.nan))  # or handle differently
    elif sol.t.shape[0] != len(t_eval):
        print(f"Unexpected t.shape: {sol.t.shape}, expected {len(t_eval)}")
        results.append(np.full_like(t_eval, np.nan))
    else:
        results.append(sol.y[0])

y = np.stack(results, axis=0)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(t_eval, y.T)
ax.set_xlabel("Time")
ax.set_ylabel("y")
ax.set_title("Bistable ODE Solution")
plt.show()

In [ ]:
import matplotlib.animation as animation
from scipy.stats import gaussian_kde

fig, ax = plt.subplots(figsize=(8, 4))
x_vals = np.linspace(0, 4, 500)

# Initial KDE plot
initial_kde = gaussian_kde(y[:, 0])
line, = ax.plot(x_vals, initial_kde(x_vals), color='blue')

ax.set_xlim(0, 4)
ax.set_ylim(0, 2.0)
ax.set_xlabel("y")
ax.set_ylabel("Density")
ax.set_title(f"KDE of y at t = {t_eval[0]:.2f}")

def update(frame):
    ax.clear()
    try:
        kde = gaussian_kde(y[:, frame])
        ax.plot(x_vals, kde(x_vals), color='blue')
    except Exception:
        ax.plot(x_vals, np.zeros_like(x_vals), color='red', linestyle='--')
    
    ax.set_xlim(0, 4)
    ax.set_ylim(0, 2.0)
    ax.set_xlabel("y")
    ax.set_ylabel("Density")
    ax.set_title(f"KDE of y at t = {t_eval[frame]:.2f}")

anime = animation.FuncAnimation(fig, update, frames=len(t_eval), interval=50)

plt.close()

HTML(anime.to_html5_video())


In [ ]:
import seaborn as sns
import matplotlib.animation as animation

fig, ax = plt.subplots(figsize=(8, 4))
x_vals = np.linspace(0, 4, 500)

def update(frame):
    ax.clear()
    sns.kdeplot(y[:, frame], bw_adjust=0.5, fill=True, ax=ax, color="skyblue")
    ax.set_xlim(0, 4)
    ax.set_ylim(0, 2.0)
    ax.set_xlabel("y")
    ax.set_ylabel("Density")
    ax.set_title(f"Smooth Density (Seaborn KDE) at t = {t_eval[frame]:.2f}")

anime = animation.FuncAnimation(fig, update, frames=len(t_eval), interval=50)
plt.close()

HTML(anime.to_html5_video())

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
bins = np.linspace(0, 4, 100)

def update(frame):
    ax.clear()
    ax.hist(y[:, frame], bins=bins, density=True, alpha=0.6, color='skyblue')
    ax.plot(y[:, frame], np.zeros_like(y[:, frame]), '|', color='black', markersize=8)
    ax.set_xlim(0, 4)
    ax.set_ylim(0, 2.0)
    ax.set_xlabel("y")
    ax.set_ylabel("Density")
    ax.set_title(f"Histogram + Rug Plot at t = {t_eval[frame]:.2f}")

anime = animation.FuncAnimation(fig, update, frames=len(t_eval), interval=50)
plt.close()

HTML(anime.to_html5_video())

# %%